In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # this is for running this notebook relative to the main directory

import asyncio
from datetime import datetime, timedelta
from architect_py.async_client import AsyncClient
from architect_py.scalars import OrderDir, TradableProduct
from architect_py.utils.nearest_tick import TickRoundMethod
from architect_py.graphql_client.enums import (
    OrderType,
    TimeInForce,
)

from architect_py.grpc_client import definitions

from decimal import Decimal

import logging

In [ ]:
api_key = ""
api_secret = ""

paper_trading = True

if not api_key or not api_secret:
    raise ValueError(
        "Please set the api_key, api_secret variables before running this script"
    )

In [ ]:
client = await AsyncClient.connect(api_key=api_key, api_secret=api_secret, paper_trading=paper_trading)

In [ ]:
await client.who_am_i()

In [ ]:
await client.list_accounts()  # if in paper trading mode and you do not see an account, then go to the web gui (app.architect.co) and then enable paper account on there before running

In [ ]:
await client.get_cme_futures_series("ES CME Futures")

In [ ]:
await client.search_symbols("ES")

In [ ]:
ES_symbol = "ES 20250620 CME Future"
NQ_symbol = "NQ 20250620 CME Future"
ES_tradable_product = TradableProduct(ES_symbol, "USD")
NQ_tradable_product = TradableProduct(NQ_symbol, "USD")

In [ ]:
await client.get_product_infos([ES_symbol, NQ_symbol])

In [ ]:
await client.get_market_status(ES_tradable_product, "CME")

In [ ]:
await client.get_execution_info(ES_tradable_product, "CME")

In [ ]:
await client.get_execution_infos([ES_tradable_product, NQ_tradable_product])

In [ ]:
await client.get_account_summaries()

In [ ]:
await client.get_all_open_orders()

In [ ]:
accounts = await client.list_accounts()
await client.get_account_summaries(accounts=[account.account.id for account in accounts])

In [ ]:
await client.get_l2_book_snapshot(venue="CME", symbol=ES_tradable_product)

In [ ]:
execution_venue = "CME"
quantity = Decimal(1)
limit_price=Decimal(5000)
account = (await client.list_accounts())[0].account.name

if not account.startswith("PAPER"):
    raise ValueError("You are NOT in a paper trading environment. This is a real order.")

order = await client.send_limit_order(
    symbol=ES_tradable_product,
    execution_venue=execution_venue,
    odir=OrderDir.BUY,
    quantity=quantity,
    limit_price=limit_price,
    account=account,
)
order